# 12) Paper 3 — Mondrian Group-Conditional Coverage

Notebook de soporte para el paper Mondrian:
- **Objetivo**: evidenciar cobertura por subgrupo y estabilidad temporal OOT.
- **Salidas**: `reports/paper_material/paper3/figures/` y `reports/paper_material/paper3/tables/`.

In [ ]:
from __future__ import annotations

import json
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"

PROJECT_ROOT = (
    Path.cwd().resolve().parent if Path.cwd().name == "notebooks" else Path.cwd().resolve()
)
DATA_DIR = PROJECT_ROOT / "data" / "processed"
MODEL_DIR = PROJECT_ROOT / "models"


def load_parquet(name: str) -> pd.DataFrame:
    return pd.read_parquet(DATA_DIR / f"{name}.parquet")


def load_json(name: str, from_models: bool = False) -> dict:
    base = MODEL_DIR if from_models else DATA_DIR
    return json.loads((base / f"{name}.json").read_text())


def ensure_dirs(base: Path) -> dict[str, Path]:
    dirs = {
        "base": base,
        "fig": base / "figures",
        "tbl": base / "tables",
    }
    for d in dirs.values():
        d.mkdir(parents=True, exist_ok=True)
    return dirs


def export_figure(fig: go.Figure, stem: str, out_fig_dir: Path) -> None:
    html_path = out_fig_dir / f"{stem}.html"
    fig.write_html(html_path)
    try:
        png_path = out_fig_dir / f"{stem}.png"
        fig.write_image(png_path, width=1400, height=850, scale=2)
        print(f"Saved: {html_path} and {png_path}")
    except Exception as exc:  # noqa: BLE001
        print(f"Saved HTML only ({html_path}). PNG skipped: {exc}")


def export_table(df: pd.DataFrame, stem: str, out_tbl_dir: Path, max_rows: int = 2000) -> None:
    csv_path = out_tbl_dir / f"{stem}.csv"
    tex_path = out_tbl_dir / f"{stem}.tex"
    out_df = df.copy().head(max_rows)
    out_df.to_csv(csv_path, index=False)
    try:
        latex = out_df.to_latex(index=False, escape=False)
        tex_path.write_text(latex, encoding="utf-8")
        print(f"Saved: {csv_path} and {tex_path}")
    except Exception as exc:  # noqa: BLE001
        print(f"Saved CSV only ({csv_path}). LaTeX skipped: {exc}")

In [ ]:
out = ensure_dirs(PROJECT_ROOT / "reports" / "paper_material" / "paper3")
pipeline_summary = load_json("pipeline_summary")
conformal_status = load_json("conformal_policy_status", from_models=True)
group_metrics = load_parquet("conformal_group_metrics_mondrian")
monthly = load_parquet("conformal_backtest_monthly")
monthly_grade = load_parquet("conformal_backtest_monthly_grade")
alerts = load_parquet("conformal_backtest_alerts")
benchmark = load_parquet("conformal_variant_benchmark")
benchmark_by_group = load_parquet("conformal_variant_benchmark_by_group")
print("group_metrics", group_metrics.shape)
print("monthly", monthly.shape)
print("monthly_grade", monthly_grade.shape)
print("alerts", alerts.shape)
print("benchmark", benchmark.shape)

In [ ]:
conformal = pipeline_summary.get("conformal", {})
key = pd.DataFrame(
    [
        {
            "metric": "coverage_90",
            "value": conformal_status.get("coverage_90", conformal.get("coverage_90", np.nan)),
        },
        {
            "metric": "coverage_95",
            "value": conformal_status.get("coverage_95", conformal.get("coverage_95", np.nan)),
        },
        {"metric": "avg_width_90", "value": conformal_status.get("avg_width_90", np.nan)},
        {
            "metric": "min_group_coverage_90",
            "value": conformal_status.get("min_group_coverage_90", np.nan),
        },
        {"metric": "checks_passed", "value": conformal_status.get("checks_passed", np.nan)},
        {"metric": "checks_total", "value": conformal_status.get("checks_total", np.nan)},
    ]
)
key

In [ ]:
# Figure 1: group coverage
fig1 = px.bar(
    group_metrics.melt(
        id_vars=["group"],
        value_vars=["coverage_90", "coverage_95"],
        var_name="metric",
        value_name="coverage",
    ),
    x="group",
    y="coverage",
    color="metric",
    barmode="group",
    title="Paper3-Fig1: Coverage by Grade",
)
fig1.add_hline(y=0.90, line_dash="dash", line_color="orange")
fig1.add_hline(y=0.95, line_dash="dot", line_color="green")
fig1
export_figure(fig1, "paper3_fig1_coverage_by_grade", out["fig"])

In [ ]:
# Figure 2: temporal coverage OOT
fig2 = px.line(
    monthly.sort_values("month"),
    x="month",
    y=["coverage_90", "coverage_95"],
    markers=True,
    title="Paper3-Fig2: Monthly Coverage OOT",
)
fig2.add_hline(y=0.90, line_dash="dash", line_color="orange")
fig2.add_hline(y=0.95, line_dash="dot", line_color="green")
fig2
export_figure(fig2, "paper3_fig2_monthly_coverage", out["fig"])

In [ ]:
# Figure 3: variant benchmark
fig3 = px.scatter(
    benchmark,
    x="avg_width",
    y="min_group_coverage",
    color="variant",
    size="coverage",
    hover_data=["coverage_gap", "std_group_coverage"],
    title="Paper3-Fig3: Variant Trade-off",
)
fig3
export_figure(fig3, "paper3_fig3_variant_tradeoff", out["fig"])

In [ ]:
# Figure 4 (appendix): month x grade heatmap for coverage_90
mg = monthly_grade.copy()
mg["month"] = pd.to_datetime(mg["month"]).dt.to_period("M").astype(str)
heat = mg.pivot_table(index="grade", columns="month", values="coverage_90", aggfunc="mean")
fig4 = px.imshow(
    heat,
    labels={"x": "Month", "y": "Grade", "color": "Coverage 90"},
    title="Paper3-Fig4: Monthly Grade Coverage Heatmap (90%)",
)
fig4
export_figure(fig4, "paper3_fig4_month_grade_heatmap", out["fig"])

In [ ]:
# Export tables
export_table(key, "paper3_table0_key_metrics", out["tbl"])
export_table(group_metrics, "paper3_table1_group_metrics", out["tbl"])
export_table(monthly, "paper3_table2_monthly_backtest", out["tbl"])
export_table(monthly_grade, "paper3_tableA1_monthly_grade", out["tbl"], max_rows=20000)
export_table(alerts, "paper3_tableA2_alerts", out["tbl"], max_rows=20000)
export_table(benchmark, "paper3_tableA3_variant_benchmark", out["tbl"])
export_table(benchmark_by_group, "paper3_tableA4_benchmark_by_group", out["tbl"], max_rows=20000)

## Threats to Validity (draft)
- Efectos de muestra finita por grupo (especialmente en grades con baja prevalencia).
- Riesgo de drift temporal y desviaciones de exchangeability.
- Dependencia de thresholds de alerta para operación.

## Reproducibilidad
```bash
uv run dvc repro generate_conformal benchmark_conformal_variants backtest_conformal_coverage validate_conformal_policy
uv run pytest -q tests/test_models/test_conformal.py
```